In [ ]:
!pip install -q boto3

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Specify the S3 Location of the Features

In [ ]:
%store -r scikit_processing_job_s3_output_prefix

In [ ]:
print('Previous Scikit Processing Job Name: {}'.format(scikit_processing_job_s3_output_prefix))

In [ ]:
prefix_train = '{}/output/raw-labeled-split-balanced-header-train'.format(scikit_processing_job_s3_output_prefix)
prefix_validation = '{}/output/raw-labeled-split-balanced-header-validation'.format(scikit_processing_job_s3_output_prefix)
prefix_test = '{}/output/raw-labeled-split-balanced-header-test'.format(scikit_processing_job_s3_output_prefix)

path_train = './{}'.format(prefix_train)
path_validation = './{}'.format(prefix_validation)
path_test = './{}'.format(prefix_test)

train_s3_uri = 's3://{}/{}'.format(bucket, prefix_train)
validation_s3_uri = 's3://{}/{}'.format(bucket, prefix_validation)
test_s3_uri = 's3://{}/{}'.format(bucket, prefix_test)

s3_input_train_data = sagemaker.s3_input(s3_data=train_s3_uri, content_type='text/csv')
s3_input_validation_data = sagemaker.s3_input(s3_data=validation_s3_uri, content_type='text/csv')
s3_input_test_data = sagemaker.s3_input(s3_data=test_s3_uri, content_type='text/csv')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

In [ ]:
!cat src_bert/bert_reviews.py

In [ ]:
from sagemaker.pytorch import PyTorch

model_output_path = 's3://{}/models/bert/script-mode/training-runs'.format(bucket)

bert_estimator = PyTorch(entry_point='bert_reviews.py',
                         source_dir='src_bert',
                         role=role,
                         train_instance_count=1, # 1 is actually faster due to communication overhead with >1
                         train_instance_type='ml.p3.8xlarge',
                         py_version='py3',
                         framework_version='1.4.0',
                         output_path=model_output_path,
                         hyperparameters={'model_type':'bert',
                                          'model_name': 'bert-base-cased',
                                          'backend': 'gloo'},
                         enable_cloudwatch_metrics=True)

### Train the model

In [ ]:
bert_estimator.fit(inputs={'train': s3_input_train_data, 
                           'validation': s3_input_validation_data,}, 
                   wait=False) 

In [ ]:
training_job_name = bert_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

In [32]:
from sagemaker.pytorch import PyTorch

bert_estimator = PyTorch.attach(training_job_name=training_job_name)

2020-03-21 18:05:40 Starting - Preparing the instances for training......
2020-03-21 18:07:00 Downloading - Downloading input data...
2020-03-21 18:07:21 Training - Downloading the training image......
2020-03-21 18:08:47 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-21 18:08:48,265 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-03-21 18:08:48,309 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-03-21 18:08:51,338 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-03-21 18:08:58,987 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-03-21 18:08:58,987 sagemaker-containers INFO     Generating setup.cfg
2020-03-21 18:08:58,988 sagemaker-containers INFO     Generating


2020-03-21 18:09:49 Uploading - Uploading generated training model#015Current iteration:  28%|██▊       | 63/225 [00:04<00:08, 18.06it/s]#033[A
#015Current iteration:  38%|███▊      | 85/225 [00:06<00:07, 18.13it/s]#033[A
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7423 sha256=f7340a8bbfcd8b09f12e24b369e721900d5471108784bc05e86ad21d2e96da02
#015Current iteration:  39%|███▊      | 87/225 [00:06<00:07, 18.15it/s]#033[A
  Stored in directory: /root/.cache/pip/wheels/1f/1b/a6/a808a7e4d1f7584e42f5e279664cd48bf24ed8392218ce6be4
#015Current iteration:  41%|████▏     | 93/225 [00:06<00:07, 18.13it/s]#033[A
  Created wheel for sacremoses: filename=sacremoses-0.0.38-py3-none-any.whl size=884629 sha256=30deccc3daff6f917b5bdc7008c85b913b234c40eeff87aa380cbe876da50757
#015Current iteration:  42%|████▏     | 95/225 [00:06<00:07, 18.13it/s]#033[A
  Stored in directory: /root/.cache/pip/wheels/03/e9/be/8b52f6e7e8c333b56f9440575b4c5eb4d96d27b5d22df5a71e
#015Current itera


2020-03-21 18:11:02 Completed - Training job completed
Training seconds: 242
Billable seconds: 242


# Load the Model

In [ ]:
# download the model artifact from AWS S3
!aws s3 cp $model_output_path/$training_job_name/output/model.tar.gz ./models/bert/

In [ ]:
import tarfile
import pickle as pkl

tar = tarfile.open('./models/bert/model.tar.gz')
tar.extractall(path='./models/bert-model')
tar.close()

In [ ]:
from simpletransformers.classification import ClassificationModel

args = {
   'fp16': False,
   'max_seq_length': 128
}

bert_model = ClassificationModel(model_type='distilbert', # bert, distilbert, etc, etc.
                                 model_name='./models/bert-model',
                                 args=args,
                                 use_cuda=False)

# Predict 


In [ ]:
predictions, raw_outputs = bert_model.predict(["""I really enjoyed this item.  I highly recommend it."""])

print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))

In [ ]:
predictions, raw_outputs = bert_model.predict(["""This item is awful and terrible."""])

print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))